In [1]:
# Import Libraries
import numpy as np
from IPython.display import display, Math
import sys
sys.path.append('../src/Code_for_Experiments')
import nci_linear_setup as ncls
import nci_polynomial_setup as ncps

## Example 1: Disjoint Pair Network with Fixed Covariates

We consider a small graph made up of disjoint connected pairs, and use fixed covariates and outcomes. 
The $\boldsymbol{\theta}_{\text{Reg}}$ coefficient is directly computed using closed-form formula.

In [2]:
#Parameters and Initialization for Example 1
n, d, p = 4, 1, 0.5
A = np.eye(n)
for i in range(0, n, 2):
    A[i, i+1] = A[i+1, i] = 1

X = np.array([[-1], [0.5], [-5], [5.5]])
X = X - np.mean(X, axis=0)
C_self = X + np.array([[1.95], [1.4], [2.3], [-2.4]])

C_cross = np.ones((n, n))

theta_Reg = np.linalg.inv(X.T @ X) @ (X.T @ np.array(C_self+1))
value = theta_Reg.item()
display(Math(fr'{{\boldsymbol{{\theta}}}}_{{\text{{Reg}}}} = {value:.4f}'))

<IPython.core.display.Math object>

In [3]:
# Variance Impact Calculation for Example 1
diff_1 = sum([1 / (p * (1 - p)) * np.sum(A[i, :]) * np.dot(theta_Reg, X[i]) ** 2 for i in range(n)])

diff_2 = 0.0
for i in range(n):
    diff_2_elem = []
    for k in range(n):
        Nik = [r for r in range(n) if A[i, r] == 1 and A[k, r] == 1 and i != k]
        ret = np.sum([2 * (1 - p) * C_cross[i, j] + 2 * C_self[i] - np.dot(theta_Reg, X[i]) for j in Nik])
        diff_2_elem.append(1 / (p * (1 - p)) * np.dot(theta_Reg, X[k]) * ret)
    diff_2 += sum(diff_2_elem)

total_diff = (diff_1 + diff_2) / n**2
print("Total variance impact (Example 1):", total_diff)

Total variance impact (Example 1): [-0.73545853]


## Example 2: Erdos-Rényi Network with Random Covariates

We generate a random network with edge probability 0.05 and use standard normal covariates. 
The outcome and weights are constructed accordingly, and we impute the estimator of $\boldsymbol{\theta}_{\text{Reg}}$, $\hat{\boldsymbol{\theta}}_{\text{Reg}}$, into the formula.

In [4]:
# Parameters and Initialization for Example 2
n, d, p = 100, 1, 0.5
A = np.eye(n)
z = np.random.binomial(n=1, p=p, size=(n,))
p_graph = 0.5
np.random.seed(14)

# Erdos-Renyi graph
for i in range(n):
    for j in range(i):
        if np.random.random() < p_graph:
            A[i][j] = A[j][i] = 1

# Random covariates
X = np.random.randn(n, d)
X = X - np.mean(X, axis=0)
C_self = np.exp(X.reshape(-1)) + np.random.randn(n)
C_cross = np.ones((n, n))

In [5]:
# Outcomes and Weights for Example 2
C_vec = np.zeros(n)
Y = np.zeros(n)
W = np.zeros(n)

for i in range(n):
    C_vec[i] = C_self[i] + np.sum((1 - p) * C_cross[i] * A[i])
    Y[i] = C_self[i] + np.sum(C_cross[i] * A[i] * z[i])
    W[i] = np.sum(A[i] * (z[i] - p)) ** 2 / (1 - p) ** 2 / p ** 2

W = np.diag(W)
theta_Reg_hat = np.linalg.inv(X.T @ W @ X) @ (X.T @ W @ Y)
value = theta_Reg_hat.item()
display(Math(fr'\hat{{\boldsymbol{{\theta}}}}_{{\text{{Reg}}}} = {value:.4f}'))

<IPython.core.display.Math object>

In [6]:
# Variance Impact Calculation for Example 2
diff_1 = sum([1 / (p * (1 - p)) * np.sum(A[i, :]) * np.dot(theta_Reg_hat, X[i]) ** 2 for i in range(n)])

diff_2 = 0.0
for i in range(n):
    diff_2_elem = []
    for k in range(n):
        Nik = [r for r in range(n) if A[i, r] == 1 and A[k, r] == 1 and i != k]
        ret = np.sum([2 * (1 - p) * C_cross[i, j] + 2 * C_self[i] - np.dot(theta_Reg_hat, X[i]) for j in Nik])
        diff_2_elem.append(1 / (p * (1 - p)) * np.dot(theta_Reg_hat, X[k]) * ret)
    diff_2 += sum(diff_2_elem)

total_diff = (diff_1 + diff_2) / n**2
print("Total variance impact (Example 1):", total_diff)

Total variance impact (Example 1): -4.14343253593864
